In [3]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.catalogImplementation":"hive"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1661646412795_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1661646412795_0004,pyspark,idle,Link,Link,None,✔


In [4]:
# required to enable explain plan
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
domaininfo = spark.sql("""select * from tecton_dev.domaininfo_snap""")
domaininfo_status = spark.sql("""select * from tecton_dev.domaininfo_status_snap""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
from pyspark.sql.functions import datediff, lower, regexp_extract

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
domaininfo = domaininfo.withColumn("tld", lower(regexp_extract("domainname", r"[.].*$", 0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
domaininfo = domaininfo.withColumn("sld", lower(regexp_extract("domainname", r"[.].*$", 0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
domaininfo = domaininfo.withColumn("create_to_update_year", datediff("updatedate", "createdate") / 365)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
domaininfo = domaininfo.withColumn("create_to_expire_year", datediff("expirationdate", "createdate") / 365)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
widen_domains = domaininfo.join(domaininfo_status, domaininfo.status == domaininfo_status.domaininfo_statusid, "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
from pyspark.sql.functions import col, row_number, when

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
from pyspark.sql.window import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
domaininfo = widen_domains.withColumn(
        "updatedate_nvl",
        when(col("updatedate").isNull(), col("modifytime"))
        .when(col("updatedate") < col("modifytime"), col("modifytime"))
        .otherwise(col("updatedate")),
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
window = Window.partitionBy("shopper_id").orderBy(col("isactiveflag").desc(), col("updatedate_nvl").desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
domaininfo = domaininfo.withColumn("row_number", row_number().over(window)).filter(col("row_number") <= 1000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
latest_domains = domaininfo.drop("row_number", "updatedate_nvl")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
domains = latest_domains

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
from pyspark.sql.functions import sum, col, countDistinct, count, lit, when

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
domains = domains.fillna(0, ["create_to_update_year", "create_to_expire_year"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
domains = domains.withColumn("domain_register_over_year_active",
                                 when((((col("create_to_update_year") > 1) | (col("create_to_expire_year") > 1))
                                       & col("isactiveflag")), 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
domains = domains.withColumn("domain_register_over_year_inactive",
                                 when((((col("create_to_update_year") > 1) | (col("create_to_expire_year") > 1))
                                       & ~col("isactiveflag")), 1).otherwise(0)
                                 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
domains = domains.withColumn("domain_register_over_year_all",
                                 col("domain_register_over_year_active") + col("domain_register_over_year_inactive"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
active_domains = domains.filter(col("isactiveflag"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
"order_feq" in active_domains.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [127]:
active_domains.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['shopper_id', 'order_id', 'tx_source_database', 'tx_source_table', 'tx_action', 'tx_write_time', 'tx_source_id', 'tx_source_time', 'id', 'privatelabelid', 'ownerid', 'domainname', 'registrationperiod', 'createdate', 'expirationdate', 'updatedate', 'status', 'renewperiod', 'modifytime', 'errordesc', 'row_id', 'agreedtocontract', 'isoingo', 'registryid', 'authinfo', 'tldid', 'sendemail', 'xfrawaydate', 'processctrl', 'lastusernote', 'modifiedby', 'internalregistrarid', 'selectedforrenewalcall', 'autorenewflag', 'repossessedforsale', 'repossessedpriceperyear', 'repossessedcategoryid', 'isproxied', 'islocked', 'gdshop_receipt_item_detailid', 'previousregistrarid', 'ccemailaddress', 'eppjobid', 'lasttransferstatusdate', 'lasttransferstatusupdatebyid', 'domxfrattempts', 'curadminemail', 'domxfrkey', 'processdate', 'processuser', 'transfersendemail', 'event_id', 'lasttransferemaildate', 'processguid', 'fraud', 'isinternaltransfer', 'parent_bundle_id', 'parent_bundle_product_typeid', 'gdshop_

In [142]:
non_duplicate_orders = active_domains.groupBy(["shopper_id", "order_id"])\
        .agg(sum(lit(1)).alias("order_freq"))\
        .select("shopper_id", "order_id", "order_freq")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
non_duplicate_orders.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------+
|shopper_id|  order_id|order_freq|
+----------+----------+----------+
| 163443605|1346991122|         1|
| 231608122|1877142280|         1|
|  23839928|2283334812|         1|
| 272998478|2283333323|         1|
| 280945054|2283334222|         1|
| 349538039|2283335176|         1|
|  37971614| 260465942|         1|
| 480577213|2270982562|         1|
| 527263904|2277082040|         2|
| 528451620|2283335036|         3|
| 410809803|2049539313|        27|
| 410809803|1973591455|         9|
| 410809803|1982235290|        13|
| 410809803|1974860483|         1|
| 102704532|1908037473|         1|
| 102704532|1900185690|         1|
| 102704532|1901848300|         1|
| 102704532|1930755800|         1|
| 102704532|1929297708|         1|
| 102704532|1910221031|         1|
+----------+----------+----------+
only showing top 20 rows

In [144]:
active_domains = active_domains.join(non_duplicate_orders, ["shopper_id", "order_id"], "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
active_domains.select([
    "shopper_id", "order_id", "order_freq"
]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------+
|shopper_id|  order_id|order_freq|
+----------+----------+----------+
| 527263904|2277082040|         2|
| 527263904|2277082040|         2|
| 480577213|2270982562|         1|
| 272998478|2283333323|         1|
| 163443605|1346991122|         1|
| 231608122|1877142280|         1|
| 280945054|2283334222|         1|
| 349538039|2283335176|         1|
| 528451620|2283335036|         3|
| 528451620|2283335036|         3|
| 528451620|2283335036|         3|
|  37971614| 260465942|         1|
|  23839928|2283334812|         1|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
| 410809803|1982235290|        13|
+----------+----------+----------+
only showing top 20 rows

In [148]:
active_domains = active_domains.withColumn("1_domain_order", when(col("order_freq") == 1, 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
grouped_active_domains = active_domains.groupBy("shopper_id").agg(
        sum("order_freq").alias("orders_count"),
        countDistinct("tld").alias("active_tlds"),
        countDistinct("sld").alias("active_slds"),
        count(lit(1)).alias("active_domains"),
        sum(col("1_domain_order")).alias("1_domain_order")
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
grouped_active_domains = grouped_active_domains.withColumn("multipurchase_count",
                                                               col("active_domains") - col("orders_count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
grouped_active_domains = grouped_active_domains.withColumn("tld_count", col("active_tlds"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
grouped_active_domains = grouped_active_domains.withColumn("duplicate_slds",
                                                               col("active_domains") - col("active_slds"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
grouped_active_domains = grouped_active_domains.withColumn("diff_tld_prop",
                                                               col("active_tlds") / col("active_domains"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
grouped_active_domains = grouped_active_domains.withColumn("duplicate_slds_prop",
                                                               col("active_slds") / col("active_domains"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
grouped_active_domains = grouped_active_domains.withColumn("average_domain_per_order",
                                                               when(col("orders_count") > 0,
                                                                    col("active_domains")/col("orders_count"))
                                                               .otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
grouped_active_domains = grouped_active_domains.withColumn("duplicate_slds_prop",
                                                               col("active_slds") / col("active_domains"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
grouped_domains = domains.groupBy("shopper_id").agg(
        sum("domain_register_over_year_active").alias("domain_register_over_year_active"),
        sum("domain_register_over_year_inactive").alias("domain_register_over_year_inactive"),
        sum("domain_register_over_year_all").alias("domain_register_over_year_all")
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
merged_domains = grouped_domains\
        .join(grouped_active_domains, "shopper_id", "left")\
        .select("shopper_id", "multipurchase_count", "tld_count", "duplicate_slds", "diff_tld_prop",
                "duplicate_slds_prop", "average_domain_per_order")\
        .fillna(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
_threshold_categorical_column_names = [
    f"{status_name}_{feature_name}" for status_name, feature_name, _ in _threshold_categorical_features
]

_scaled_numeric_column_names = [
    f"{status_name}_{feature_name}" for status_name, feature_name, _ in _scaled_numeric_features
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
cols = [
    "shopper_id", "domain_register_over_year_active", "domain_register_over_year_inactive",
    "domain_register_over_year_all", "domain_register_over_year_all", "orders_count", "tld_count",
    "active_slds", "active_domains", "1_domain_order", "multipurchase_count", "duplicate_slds", "diff_tld_prop",
    "duplicate_slds_prop", "average_domain_per_order", "duplicate_slds_prop"
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
merged_domains.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+---------+--------------+------------------+-------------------+------------------------+
|shopper_id|multipurchase_count|tld_count|duplicate_slds|     diff_tld_prop|duplicate_slds_prop|average_domain_per_order|
+----------+-------------------+---------+--------------+------------------+-------------------+------------------------+
| 163443605|                  0|        1|             0|               1.0|                1.0|                     1.0|
| 231608122|                  0|        1|             0|               1.0|                1.0|                     1.0|
|  23839928|                  0|        1|             0|               1.0|                1.0|                     1.0|
| 272998478|                  0|        1|             0|               1.0|                1.0|                     1.0|
| 280945054|                  0|        1|             0|               1.0|                1.0|                     1.0|
| 349538039|            

In [177]:
from datetime import datetime
mock_domaininfo_txlog_columns = [
    "shopper_id",
    "tx_source_time",
    "tx_date",
]

mock_domaininfo_txlog_data = [
    ["mock_shopper_id", datetime(2022, 8, 1), datetime(2022, 8, 1)],
    ["before_materialization_time", datetime(2022, 1, 1), datetime(2022, 1, 2)],
    ["after_materialization_time", datetime(2022, 12, 1), datetime(2022, 12, 1)],
]

mock_domaininfo_status_columns = ["domaininfo_statusid", "isactiveflag"]
mock_domaininfo_status_data = [
    [1, True],
    [2, False],
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
mock_domaininfo_columns = [
    "shopper_id",
    "status",
    "previousregistrarid",
    "domainname",
    "gaining_registrar_id",
    "modifytime",
    "updatedate",
    "createdate",
    "expirationdate",
    "deletedate",
    "canceleddate",
    "autorenewflag",
    "billingstatus",
    "islocked",
    "isproxied",
    "isinternaltransfer",
    "islimited",
    "isregistrarhold",
    "issuperlocked",
    "lasttransferstatusdate",
    "istransferprotected",
    "isexpirationprotected",
    "issmartdomain",
    "invalidwhois",
    "order_id"
]
mock_domaininfo_data = [
    [
        "before_materialization_time",
        1,
        1,
        "abcd.mock",
        1,
        datetime(2022, 1, 1),
        datetime(2022, 1, 2),
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        datetime(2022, 1, 1),  # lasttransferstatusdate
        None,
        None,
        None,
        None,
        111
    ],
    [
        "mock_shopper_id",
        1,
        1,
        "abcd.com",
        1,
        datetime(2022, 8, 1),
        datetime(2022, 8, 2),
        datetime(2022, 1, 1),
        datetime(2023, 1, 1),
        None,
        None,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        datetime(2022, 1, 1),  # lasttransferstatusdate
        False,
        False,
        False,
        False,
        222
    ],
    [
        "mock_shopper_id",
        2,
        -1,
        "abcd.info",
        None,
        datetime(2022, 8, 2),
        datetime(2022, 8, 1),
        datetime(2022, 8, 1),
        datetime(2022, 8, 1),
        datetime(2022, 8, 1),
        datetime(2022, 8, 1),
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        datetime(2022, 1, 1),  # lasttransferstatusdate
        False,
        False,
        False,
        False,
        111
    ],
    [
        "after_materialization_time",
        2,
        None,
        "wxyz.mock",
        None,
        datetime(2022, 12, 1),
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        datetime(2022, 1, 1),  # lasttransferstatusdate
        None,
        None,
        None,
        None,
        111
    ],
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [194]:
df_status = pd.DataFrame(data=mock_domaininfo_status_data, columns=mock_domaininfo_status_columns)
df_tx_log = pd.DataFrame(data=mock_domaininfo_txlog_data, columns=mock_domaininfo_txlog_columns)
df_info = pd.DataFrame(data=mock_domaininfo_data, columns=mock_domaininfo_columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
df_status.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   domaininfo_statusid  isactiveflag
0                    1          True
1                    2         False

In [196]:
df_tx_log.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                    shopper_id tx_source_time    tx_date
0              mock_shopper_id     2022-08-01 2022-08-01
1  before_materialization_time     2022-01-01 2022-01-02
2   after_materialization_time     2022-12-01 2022-12-01

In [197]:
df_info.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                    shopper_id  status  ...  invalidwhois order_id
0  before_materialization_time       1  ...          None      111
1              mock_shopper_id       1  ...         False      222
2              mock_shopper_id       2  ...         False      111
3   after_materialization_time       2  ...          None      111

[4 rows x 25 columns]

In [198]:
required_cols = [
    "shopper_id", "domainname", "updatedate", "expirationdate", "createdate", "order_id", 
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [199]:
trimmed_df = df_info[required_cols]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
trimmed_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                    shopper_id domainname  ... createdate order_id
0  before_materialization_time  abcd.mock  ...        NaT      111
1              mock_shopper_id   abcd.com  ... 2022-01-01      222
2              mock_shopper_id  abcd.info  ... 2022-08-01      111
3   after_materialization_time  wxyz.mock  ...        NaT      111

[4 rows x 6 columns]